In [ ]:
# default_exp node
# hide
_FNAME='node'

import unittest
from unittest import mock
from nbdev.export import notebook2script
import os
TESTCASE = unittest.TestCase()
_nbpath = os.path.join(_dh[0], _FNAME+'.ipynb')
notebook2script(_nbpath)

Converted node.ipynb.


In [ ]:
#export
import json
import mranderson.db as db
from mranderson.db import Query, MATCH, RETURN, NoSearchResults


In [ ]:
from mranderson.sandbox import start_neo4j, start_fresh, create_totem
container_name = 'mranderson_node_unittest'
container, driver = start_neo4j(container_name=container_name)

In [ ]:
#export

#alias node.txt as db.node as a convenience
txt = db.node

def create(node_text:str):
    '''
    Creates the node described in node_text
    '''
    q = Query()
    q.add("CREATE")
    q.add(node_text)
    return q.create()

def exists(*args, **kwargs) -> bool:
    '''
    args and kwargs are passed along to `node`
    Returns a bool on whether or not this node exists in the database
    If several nodes exist, you still only get `True`
    '''
    q = Query()
    q.add("MATCH")
    q.add(db.node(*args, variable='var', **kwargs))
    q.add('RETURN count(var) as num')
    #use single here because count always returns a single value
    #this query just returns the number of nodes as a number, not the nodes themselves
    resp = q.single()
    return resp > 0

def set_properties(label:str, search_properties:dict, node_properties:dict=None):
    '''
    Find a node of and set properties to it.  The node must exist or this will fail
    label: string of the node Label
    search_properties: dict of properties the existing node must have.  e.g. eid="abc123"
    additional kwargs will be applied to the node
    '''
    query = Query()
    var = 'var'
    query.add(MATCH(label, var, **search_properties))
    for prop, val in node_properties.items():
        query.add('SET var.{prop}={jsonval}'.format(
                    prop=prop,
                    jsonval=json.dumps(val)))
    resp = query.create()
    return resp

def ensure(label:str, search_properties:dict, node_properties:dict=None):
    '''
    Ensure a node with search_properties exists.  If it does not, then create it.
    Finally, set the nodes properties to node_properties.  It should retain all vital_properties.
    
    If the node existed before and had properties, those properties are unchanged unless they
    are included in node_properties.
    '''
    node_vitals = db.node(label, 'node', **search_properties)
    node_properties = {} if node_properties is None else node_properties
    
    if not exists(label, **search_properties):
        create(node_vitals)
    
    if node_properties:
        set_properties(label, search_properties, node_properties)
        

In [ ]:
def test_ensure():
    TESTCASE.assertTrue(start_fresh())
    create_totem()
    TESTCASE.assertTrue(exists('Totem'))

    TESTCASE.assertFalse(exists('Test'))
    ensure('Test', search_properties={'id': 1}, node_properties={'value': 2})
    TESTCASE.assertTrue(exists('Test'))
    
    ensure('Test', search_properties={'id': 1}, node_properties={'othervalue': 3})
    data = db.fetch('Test', id=1)
    TESTCASE.assertDictEqual(data, {'id': 1, 'value': 2, 'othervalue': 3})
    print(data)
    
    print("SUCCESS")
test_ensure()

{'id': 1, 'value': 2, 'othervalue': 3}
SUCCESS


In [ ]:
#export
def get_properties(label, properties, **kwargs):
    '''
    label: str
        The label of the node youre looking for.  e.g. Equipment
    properties: iterable
        A list or tuple of the strings of property names
    Any additional kwargs will go to searching for the node.  e.g. {"eid": "device_eid"}
    Returns list of the properties of the node.  Node must be unique.
    '''
    query = Query()
    query.add(MATCH(label, 'var', **kwargs))
    returnprops = ['var.{}'.format(prop) for prop in properties]
    query.add(RETURN(','.join(returnprops)))

    try:
        resp = query.only()
    except NoSearchResults:
        return []
    return [resp['var.{}'.format(prop)] for prop in properties]

In [ ]:
container.stop()